In [2]:
# Employee Management System - Complete Solution
# Run this entire cell in Google Colab

# Import required libraries
import ipywidgets as widgets
from IPython.display import display, clear_output
import sqlite3
import pandas as pd

# Initialize database
conn = sqlite3.connect(':memory:')  # Using in-memory DB (change to 'employees.db' for persistent storage)
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        emp_id TEXT PRIMARY KEY,
        name TEXT NOT NULL,
        mobile TEXT,
        salary REAL
    )
''')
conn.commit()

# Add sample data
sample_data = [
    ('EMP001', 'John Doe', '1234567890', 50000),
    ('EMP002', 'Jane Smith', '9876543210', 60000)
]
cursor.executemany("INSERT OR IGNORE INTO employees VALUES (?, ?, ?, ?)", sample_data)
conn.commit()

# Create UI components
# Input fields
emp_id = widgets.Text(placeholder='EMP001', description='Employee ID:', style={'description_width': '100px'})
name = widgets.Text(placeholder='John Doe', description='Full Name:', style={'description_width': '100px'})
mobile = widgets.Text(placeholder='1234567890', description='Mobile:', style={'description_width': '100px'})
salary = widgets.BoundedFloatText(value=0, min=0, max=1000000, step=1000, description='Salary:',
                                style={'description_width': '100px'})

# Buttons with icons
submit_btn = widgets.Button(description="\u2714 Submit", button_style='success')
update_btn = widgets.Button(description="\u270E Update", button_style='info')
delete_btn = widgets.Button(description="\u2716 Delete", button_style='danger')
clear_btn = widgets.Button(description="\u232B Clear", button_style='warning')
refresh_btn = widgets.Button(description="\u27F3 Refresh")

# Output area
output = widgets.Output()

# Database display function
def show_database():
    with output:
        clear_output(wait=True)
        df = pd.read_sql_query("SELECT * FROM employees", conn)
        display(df)

# Button handlers
def submit_data(b):
    with output:
        if not all([emp_id.value, name.value, mobile.value]):
            print("❌ Error: All fields are required!")
            return

        try:
            cursor.execute(
                "INSERT INTO employees VALUES (?, ?, ?, ?)",
                (emp_id.value, name.value, mobile.value, salary.value)
            )
            conn.commit()
            print(f"✅ Added employee {emp_id.value} successfully!")
            show_database()
            clear_fields()
        except sqlite3.IntegrityError:
            print(f"❌ Error: Employee ID {emp_id.value} already exists!")

def update_data(b):
    with output:
        if not emp_id.value:
            print("❌ Error: Employee ID required for update")
            return

        try:
            cursor.execute(
                "UPDATE employees SET name=?, mobile=?, salary=? WHERE emp_id=?",
                (name.value, mobile.value, salary.value, emp_id.value)
            )
            if cursor.rowcount == 0:
                print(f"❌ Error: Employee ID {emp_id.value} not found")
            else:
                conn.commit()
                print(f"✅ Updated employee {emp_id.value} successfully!")
                show_database()
        except Exception as e:
            print(f"❌ Error: {str(e)}")

def delete_data(b):
    with output:
        if not emp_id.value:
            print("❌ Error: Employee ID required for deletion")
            return

        cursor.execute("DELETE FROM employees WHERE emp_id=?", (emp_id.value,))
        if cursor.rowcount == 0:
            print(f"❌ Error: Employee ID {emp_id.value} not found")
        else:
            conn.commit()
            print(f"✅ Deleted employee {emp_id.value} successfully!")
            show_database()
            clear_fields()

def clear_fields(b=None):
    emp_id.value = ""
    name.value = ""
    mobile.value = ""
    salary.value = 0

# Assign button handlers
submit_btn.on_click(submit_data)
update_btn.on_click(update_data)
delete_btn.on_click(delete_data)
clear_btn.on_click(clear_fields)
refresh_btn.on_click(lambda b: show_database())

# Create and display the UI
form = widgets.VBox([
    widgets.HTML("<h2>Employee Management System</h2>"),
    emp_id,
    name,
    mobile,
    salary,
    widgets.HBox([submit_btn, update_btn, delete_btn, clear_btn, refresh_btn]),
    output
])

display(form)
show_database()  # Show initial data

In [1]:
# Employee Management System with Gradio - Complete Working Version
# Run this entire code in one Colab cell

!pip install gradio pandas

import gradio as gr
import sqlite3
import pandas as pd
from datetime import datetime

# ========== DATABASE SETUP ==========
def init_db():
    conn = sqlite3.connect('employees.db')
    cursor = conn.cursor()

    # Drop table if it exists to avoid schema conflicts
    cursor.execute("DROP TABLE IF EXISTS employees")

    # Create table with proper syntax
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS employees (
            emp_id TEXT PRIMARY KEY,
            name TEXT NOT NULL,
            mobile TEXT,
            salary REAL,
            department TEXT,
            join_date TEXT
        )
    ''')

    # Insert sample data if empty
    cursor.execute("SELECT COUNT(*) FROM employees")
    if cursor.fetchone()[0] == 0:
        sample_data = [
            ('EMP001', 'Alex Johnson', '555-0101', 75000, 'Engineering', '2023-01-15'),
            ('EMP002', 'Maria Garcia', '555-0102', 68000, 'Marketing', '2023-02-20')
        ]
        cursor.executemany(
            "INSERT INTO employees VALUES (?, ?, ?, ?, ?, ?)",
            sample_data
        )
    conn.commit()
    return conn

# Initialize database
conn = init_db()

# ========== DATABASE OPERATIONS ==========
def get_employees():
    return pd.read_sql_query("SELECT * FROM employees ORDER BY emp_id", conn)

def add_employee(emp_id, name, mobile, salary, department, join_date):
    try:
        conn.execute(
            "INSERT INTO employees VALUES (?, ?, ?, ?, ?, ?)",
            (emp_id, name, mobile, float(salary), department, join_date)
        )
        conn.commit()
        return True, get_employees()
    except sqlite3.IntegrityError:
        return False, "⚠️ Employee ID already exists!"
    except Exception as e:
        return False, f"⚠️ Error: {str(e)}"

def update_employee(emp_id, name, mobile, salary, department, join_date):
    try:
        conn.execute(
            """UPDATE employees
            SET name=?, mobile=?, salary=?, department=?, join_date=?
            WHERE emp_id=?""",
            (name, mobile, float(salary), department, join_date, emp_id)
        )
        conn.commit()
        return True, get_employees()
    except Exception as e:
        return False, f"⚠️ Error: {str(e)}"

def delete_employee(emp_id):
    try:
        conn.execute("DELETE FROM employees WHERE emp_id=?", (emp_id,))
        conn.commit()
        return True, get_employees()
    except Exception as e:
        return False, f"⚠️ Error: {str(e)}"

def load_employee(emp_id):
    cursor = conn.execute("SELECT * FROM employees WHERE emp_id=?", (emp_id,))
    result = cursor.fetchone()
    if result:
        return {
            "name": result[1],
            "mobile": result[2],
            "salary": result[3],
            "department": result[4],
            "join_date": result[5]
        }
    return None

# ========== GRADIO INTERFACE ==========
with gr.Blocks(theme=gr.themes.Soft(), title="Employee Management") as app:
    gr.Markdown("# 🏢 Employee Management System")
    gr.Markdown("Manage your organization's employee records with this interactive dashboard")

    # View Employees Tab
    with gr.Tab("👥 View All Employees"):
        employee_table = gr.Dataframe(
            value=get_employees(),
            interactive=False,
            wrap=True
        )
        refresh_btn = gr.Button("🔄 Refresh Data")
        refresh_btn.click(fn=get_employees, outputs=employee_table)

    # Add Employee Tab
    with gr.Tab("➕ Add New Employee"):
        with gr.Row():
            new_id = gr.Textbox(label="Employee ID*", placeholder="EMP123")
            new_name = gr.Textbox(label="Full Name*", placeholder="John Doe")
        with gr.Row():
            new_mobile = gr.Textbox(label="Mobile*", placeholder="555-0100")
            new_salary = gr.Number(label="Salary*", value=50000)
        with gr.Row():
            new_dept = gr.Dropdown(
                ["Engineering", "Marketing", "HR", "Finance", "Operations", "Sales"],
                label="Department*",
                value="Engineering"
            )
            new_date = gr.Textbox(label="Join Date*", placeholder="YYYY-MM-DD", value=datetime.today().strftime('%Y-%m-%d'))

        add_status = gr.Textbox(label="Operation Status", interactive=False)
        add_btn = gr.Button("💾 Save Employee", variant="primary")
        add_btn.click(
            fn=add_employee,
            inputs=[new_id, new_name, new_mobile, new_salary, new_dept, new_date],
            outputs=[add_status, employee_table]
        )

    # Edit Employee Tab
    with gr.Tab("✏️ Edit Employee"):
        edit_id = gr.Dropdown(
            choices=list(get_employees()['emp_id']),
            label="Select Employee*",
            interactive=True
        )
        with gr.Row():
            edit_name = gr.Textbox(label="Full Name*")
            edit_mobile = gr.Textbox(label="Mobile*")
        with gr.Row():
            edit_salary = gr.Number(label="Salary*")
            edit_dept = gr.Dropdown(
                ["Engineering", "Marketing", "HR", "Finance", "Operations", "Sales"],
                label="Department*"
            )
        edit_date = gr.Textbox(label="Join Date*", placeholder="YYYY-MM-DD")

        edit_status = gr.Textbox(label="Operation Status", interactive=False)
        with gr.Row():
            update_btn = gr.Button("🔄 Update Record", variant="primary")
            delete_btn = gr.Button("🗑️ Delete Employee", variant="stop")

        # Auto-fill form when employee is selected
        edit_id.change(
            fn=load_employee,
            inputs=edit_id,
            outputs=[edit_name, edit_mobile, edit_salary, edit_dept, edit_date]
        )

        # Set up button actions
        update_btn.click(
            fn=update_employee,
            inputs=[edit_id, edit_name, edit_mobile, edit_salary, edit_dept, edit_date],
            outputs=[edit_status, employee_table]
        )
        delete_btn.click(
            fn=delete_employee,
            inputs=edit_id,
            outputs=[edit_status, employee_table]
        )

# Launch the application
app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b930130fdaf7835ab8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
